# Command line options

SoS allows you to define parameters that accept values from command line options easily. 

## Optional arguments

Any variable assignment prefixed with `parameter:` can accept values from command  line. The format of such lines are

```
# comment
parameter: var_name = default_value
```

The default value can be number, string, list of string, or expressions that return values of these types. Other types can be used as long as they can be converted to these types from user-provided values. For example

In [1]:
# path to tool gatk
parameter: gatk_path = '~/bin/GATK'

defines a variable `gatk_path` with default value `'~/bin/GATK'`.

If you execute a workflow without defining `gatk_path`, the default value will be used

In [2]:
%run
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

~/bin/GATK


You can set this parameter from command line using option `--gatk_path`

In [3]:
%run --gatk_path /my/GATK
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

/my/GATK


You can also use `-` in place of `_` in command line, although the variable name cannot have `-` in it.

In [4]:
%run --gatk-path /my/GATK
parameter: gatk_path = '~/bin/GATK'
print(gatk_path)

/my/GATK


The default value of the parameter determines the type of values it accepts. For example, the following command yields an error because the parameter expects an integer (from default value `0`) but a string is passed.

In [5]:
%sandbox --expect-error

%run --cutoff zero
parameter: cutoff = 0

Failed to process statement if "sos_handle_parameter_" in ...ff',  0\n)\n: argument --cutoff: invalid int value: 'zero'
Sandbox execution failed.

Simiarly, a list would be created if the parameter has a default value of type list

In [6]:
%run --sample-names A
parameter: sample_names = []
print(sample_names)

['A']


Finally, the convention to specify `True` or `False` of a boolean parameter is to use `--no-PARAM` for `PARAM=False`. For example,

In [7]:
%run --no-trim
parameter: trim=True
print(trim)

False


## Required arguments

In cases where there is no suitable default values and/or command line arguments are mandatary, you can list the type of arguments (e.g. `int`, `bool`, `str`, `list` of strings) in place of default values. This will force SoS to get a parameter from command line, and yield an error if no value is specified for this parameter.

For example, if an integer parameter `cutoff` is required, you can define it as

In [8]:
%sandbox --expect-error

# cutoff value
parameter: cutoff = int

Failed to process statement if "sos_handle_parameter_" in ...',  int\n)\n: Argument cutoff of type <class 'int'> is required
Sandbox execution failed.

and execute the workflow without parameter, SoS would fail with an error message. This will force the users to provide an integer to this parameter. You can do the same for lists but SoS assumes that you need a **list of strings**. For example, the following definition

In [9]:
%run --bam-files A.bam B.bam
# input bam files
parameter: bam_files = list
print(bam_files)

['A.bam', 'B.bam']


request a list of strings from command line. SoS will return a list even if only one value is provided.

## Redefinition of `parameter`

A very important property of the `parameter` directive is that **parameter definition is only effective for undefined variable**. That is to say, the parameter statement will be ignored if a parameter has been specified in some way, most likely as a variable in a step.

For example, when the `default` workflow of the following script is executed, `cutoff` takes its default value of `10`. 

In [10]:
[default_1]
parameter: cutoff=10
sh:
  echo ${cutoff}

[batch]
for cutoff in range(3):
    sos_run('default')

10


A command line argument can be used to set this parameter to `8`,

In [11]:
%rerun --cutoff 8

8


Now, if we execute another workflow in this script and calls `default` as a nested workflow, `cutoff` will use the existing value set by step `batch`, instead of using its default value

In [12]:
%rerun batch 

10
10
10


Well, not exactly. As you can see the default value of parameter `cutoff` is still used. This is actually one of the few differences in running SoS script in batch mode and in interactive mode. In interactive mode parameters will always be processed because it is quite likely that a script could be re-executed with different parameters, while having old values from previous runs. The batch mode does not have this problem (each run would start from a fresh namespace) and enforces this rule.

To see how this script behave in batch mode, you can run it as a complete script as follows.

In [13]:
[default: sigil=None]
run: args='batch -v1'
    #!/usr/bin/env sos-runner
    [default_1]
    parameter: cutoff=10
    sh:
      echo ${cutoff}

    [batch]
    for cutoff in range(3):
        sos_run('default')

0
1
2


Here `sigil=None` is used to stop string interpolation so that the script can be saved in verbatim to a file. With the `run` action and shebang line, the script itself bvecomes the command and is executed with arguments `batch -v1`. Please check section [SoS Step](SoS_Step.html) for details about the usage of this `run` action.